In [3]:
import smbus
import time

class mpu6050:
    GRAVITIY_MS2 = 9.80665
    address = None
    bus = None

    ACCEL_SCALE_MODIFIER_2G = 16384.0
    ACCEL_SCALE_MODIFIER_4G = 8192.0
    ACCEL_SCALE_MODIFIER_8G = 4096.0
    ACCEL_SCALE_MODIFIER_16G = 2048.0

    GYRO_SCALE_MODIFIER_250DEG = 131.0
    GYRO_SCALE_MODIFIER_500DEG = 65.5
    GYRO_SCALE_MODIFIER_1000DEG = 32.8
    GYRO_SCALE_MODIFIER_2000DEG = 16.4

    ACCEL_RANGE_2G = 0x00
    ACCEL_RANGE_4G = 0x08
    ACCEL_RANGE_8G = 0x10
    ACCEL_RANGE_16G = 0x18

    GYRO_RANGE_250DEG = 0x00
    GYRO_RANGE_500DEG = 0x08
    GYRO_RANGE_1000DEG = 0x10
    GYRO_RANGE_2000DEG = 0x18

    FILTER_BW_256=0x00
    FILTER_BW_188=0x01
    FILTER_BW_98=0x02
    FILTER_BW_42=0x03
    FILTER_BW_20=0x04
    FILTER_BW_10=0x05
    FILTER_BW_5=0x06

    PWR_MGMT_1 = 0x6B
    PWR_MGMT_2 = 0x6C

    ACCEL_XOUT0 = 0x3B
    ACCEL_YOUT0 = 0x3D
    ACCEL_ZOUT0 = 0x3F

    TEMP_OUT0 = 0x41

    GYRO_XOUT0 = 0x43
    GYRO_YOUT0 = 0x45
    GYRO_ZOUT0 = 0x47

    ACCEL_CONFIG = 0x1C
    GYRO_CONFIG = 0x1B
    MPU_CONFIG = 0x1A

    def __init__(self, address, bus=1):
        self.address = address
        self.bus = smbus.SMBus(bus)
        self.bus.write_byte_data(self.address, self.PWR_MGMT_1, 0x00)

    def read_i2c_word(self, register):
        high = self.bus.read_byte_data(self.address, register)
        low = self.bus.read_byte_data(self.address, register + 1)
        value = (high << 8) + low
        return value - 65536 if value >= 0x8000 else value

    def get_temp(self):
        raw_temp = self.read_i2c_word(self.TEMP_OUT0)
        return (raw_temp / 340.0) + 36.53

    def read_accel_range(self, raw = False):
        raw_data = self.bus.read_byte_data(self.address, self.ACCEL_CONFIG)
        if raw: return raw_data
        return {self.ACCEL_RANGE_2G: 2, self.ACCEL_RANGE_4G: 4, self.ACCEL_RANGE_8G: 8, self.ACCEL_RANGE_16G: 16}.get(raw_data, -1)

    def get_accel_data(self, g = False):
        x = self.read_i2c_word(self.ACCEL_XOUT0)
        y = self.read_i2c_word(self.ACCEL_YOUT0)
        z = self.read_i2c_word(self.ACCEL_ZOUT0)
        accel_range = self.read_accel_range(True)
        modifier = {
            self.ACCEL_RANGE_2G: self.ACCEL_SCALE_MODIFIER_2G,
            self.ACCEL_RANGE_4G: self.ACCEL_SCALE_MODIFIER_4G,
            self.ACCEL_RANGE_8G: self.ACCEL_SCALE_MODIFIER_8G,
            self.ACCEL_RANGE_16G: self.ACCEL_SCALE_MODIFIER_16G,
        }.get(accel_range, self.ACCEL_SCALE_MODIFIER_2G)
        x, y, z = x/modifier, y/modifier, z/modifier
        if g:
            return {'x': x, 'y': y, 'z': z}
        else:
            return {'x': x*self.GRAVITIY_MS2, 'y': y*self.GRAVITIY_MS2, 'z': z*self.GRAVITIY_MS2}

    def read_gyro_range(self, raw = False):
        raw_data = self.bus.read_byte_data(self.address, self.GYRO_CONFIG)
        if raw: return raw_data
        return {self.GYRO_RANGE_250DEG: 250, self.GYRO_RANGE_500DEG: 500,
                self.GYRO_RANGE_1000DEG: 1000, self.GYRO_RANGE_2000DEG: 2000}.get(raw_data, -1)

    def get_gyro_data(self):
        x = self.read_i2c_word(self.GYRO_XOUT0)
        y = self.read_i2c_word(self.GYRO_YOUT0)
        z = self.read_i2c_word(self.GYRO_ZOUT0)
        gyro_range = self.read_gyro_range(True)
        modifier = {
            self.GYRO_RANGE_250DEG: self.GYRO_SCALE_MODIFIER_250DEG,
            self.GYRO_RANGE_500DEG: self.GYRO_SCALE_MODIFIER_500DEG,
            self.GYRO_RANGE_1000DEG: self.GYRO_SCALE_MODIFIER_1000DEG,
            self.GYRO_RANGE_2000DEG: self.GYRO_SCALE_MODIFIER_2000DEG,
        }.get(gyro_range, self.GYRO_SCALE_MODIFIER_250DEG)
        return {'x': x/modifier, 'y': y/modifier, 'z': z/modifier}

    def get_all_data(self):
        return {
            'temperature': self.get_temp(),
            'accel': self.get_accel_data(),
            'gyro': self.get_gyro_data()
        }


In [3]:
# Tạo file rỗng tên là data_imu.txt
open("data_imu.txt", "w").close()


In [ ]:
import matplotlib.pyplot as plt
from collections import deque
import time

sensor = mpu6050(address=0x68)

buffer_size = 100
times = deque(maxlen=buffer_size)
accel_x = deque(maxlen=buffer_size)
accel_y = deque(maxlen=buffer_size)
accel_z = deque(maxlen=buffer_size)

start_time = time.time()

plt.ion()  # Bật chế độ interactive mode cho matplotlib
fig, ax = plt.subplots()
line_x, = ax.plot([], [], label='Accel X (g)')
line_y, = ax.plot([], [], label='Accel Y (g)')
line_z, = ax.plot([], [], label='Accel Z (g)')

ax.set_ylim(-2, 2)
ax.set_xlabel('Time (s)')
ax.set_ylabel('Acceleration (g)')
ax.legend()
ax.grid(True)

while True:
    current_time = time.time() - start_time
    data = sensor.get_accel_data(g=True)

    times.append(current_time)
    accel_x.append(data['x'])
    accel_y.append(data['y'])
    accel_z.append(data['z'])

    # Cập nhật trục X để luôn hiện data gần nhất
    if len(times) > 1:
        ax.set_xlim(times[0], times[-1])

    line_x.set_data(times, accel_x)
    line_y.set_data(times, accel_y)
    line_z.set_data(times, accel_z)

    fig.canvas.draw()
    fig.canvas.flush_events()

    time.sleep(0.1)  # delay 100ms giữa các lần lấy dữ liệu


In [2]:
import time
import math
import datetime


sensor = mpu6050(0x68, bus=0)

# Tỉ lệ lọc (giữa gyro và accel)
alpha = 0.9

pitch = 0
roll = 0
dt = 0.05  # Thời gian giữa 2 lần đọc, đơn vị: giây

# Ghi tiêu đề (nếu lần đầu ghi)
with open("data_imu.txt", "w") as f:
    f.write(f"{'Time':<10}  {'Ax':<10}{'Ay':<10}{'Az':<10}{'Gx':<10}{'Gy':<10}{'Gz':<10}{'Pitch':<10}{'Roll':<10}\n")

while True:
    accel_data = sensor.get_accel_data()
    gyro_data = sensor.get_gyro_data()

    # Lấy dữ liệu
    ax, ay, az = accel_data['x'], accel_data['y'], accel_data['z']
    gx, gy, gz = gyro_data['x'], gyro_data['y'], gyro_data['z']

    # Tính pitch/roll từ gia tốc kế (thô)
    accel_pitch = math.degrees(math.atan2(ay, math.sqrt(ax*ax + az*az)))
    accel_roll = math.degrees(math.atan2(-ax, az))

    # Tích phân gyro để lấy góc quay
    pitch = alpha * (pitch + gy * dt) + (1 - alpha) * accel_pitch
    roll = alpha * (roll + gx * dt) + (1 - alpha) * accel_roll

    # Lấy thời gian hiện tại dưới dạng HH:MM:SS
    current_time = datetime.datetime.now().strftime('%H:%M:%S')

    # Hiển thị lên màn hình
    print(f"[{current_time}]  Ax={ax:.2f}, Ay={ay:.2f}, Az={az:.2f} | Gx={gx:.2f}, Gy={gy:.2f}, Gz={gz:.2f} | Pitch={pitch:.2f}, Roll={roll:.2f}")

    # Ghi dữ liệu
    with open("data_imu.txt", "a") as f:
        f.write(f"{current_time:<10}  {ax:<10.2f}{ay:<10.2f}{az:<10.2f}{gx:<10.2f}{gy:<10.2f}{gz:<10.2f}{pitch:<10.2f}{roll:<10.2f}\n")

    time.sleep(dt)


NameError: name 'mpu6050' is not defined

In [ ]:
import time
import math
import datetime


# Khởi tạo cảm biến
sensor = mpu6050(0x68, bus=0)

# === Hiệu chỉnh offset gyro ===
def calibrate_gyro(sensor, samples=100):
    print("Đang hiệu chỉnh gyro... Giữ IMU cố định.")
    gx_offset = 0
    gy_offset = 0
    gz_offset = 0
    for _ in range(samples):
        data = sensor.get_gyro_data()
        gx_offset += data['x']
        gy_offset += data['y']
        gz_offset += data['z']
        time.sleep(0.01)
    print("Hoàn tất hiệu chỉnh.")
    return gx_offset / samples, gy_offset / samples, gz_offset / samples

gx_offset, gy_offset, gz_offset = calibrate_gyro(sensor)

# Tỉ lệ lọc (giữa gyro và accel)
alpha = 0.9
pitch = 0
roll = 0
dt = 0.05  # thời gian giữa 2 lần đọc

# Ghi tiêu đề
with open("data_imu.txt", "w") as f:
    f.write(f"{'Time':<10}  {'Ax':<10}{'Ay':<10}{'Az':<10}{'Gx':<10}{'Gy':<10}{'Gz':<10}{'Pitch':<10}{'Roll':<10}\n")

while True:
    accel_data = sensor.get_accel_data()
    gyro_data = sensor.get_gyro_data()

    # Trừ offset gyro
    gx = gyro_data['x'] - gx_offset
    gy = gyro_data['y'] - gy_offset
    gz = gyro_data['z'] - gz_offset

    # Dữ liệu accel
    ax = accel_data['x']
    ay = accel_data['y']
    az = accel_data['z']

    # Tính pitch/roll từ gia tốc kế
    accel_pitch = math.degrees(math.atan2(ay, math.sqrt(ax * ax + az * az)))
    accel_roll = math.degrees(math.atan2(-ax, az))

    # Bộ lọc kết hợp (complementary filter)
    pitch = alpha * (pitch + gy * dt) + (1 - alpha) * accel_pitch
    roll = alpha * (roll + gx * dt) + (1 - alpha) * accel_roll

    # Lấy thời gian hiện tại dạng HH:MM:SS
    current_time = datetime.datetime.now().strftime('%H:%M:%S')

    # Hiển thị
    print(f"[{current_time}]  Ax={ax:.2f}, Ay={ay:.2f}, Az={az:.2f} | Gx={gx:.2f}, Gy={gy:.2f}, Gz={gz:.2f} | Pitch={pitch:.2f}, Roll={roll:.2f}")

    # Ghi dữ liệu ra file
    with open("data_imu.txt", "a") as f:
        f.write(f"{current_time:<10}  {ax:<10.2f}{ay:<10.2f}{az:<10.2f}{gx:<10.2f}{gy:<10.2f}{gz:<10.2f}{pitch:<10.2f}{roll:<10.2f}\n")

    time.sleep(dt)


Đang hiệu chỉnh gyro... Giữ IMU cố định.
Hoàn tất hiệu chỉnh.
[23:43:54]  Ax=-0.57, Ay=-0.06, Az=9.98 | Gx=1.88, Gy=-0.20, Gz=-0.47 | Pitch=-0.04, Roll=0.41
[23:43:54]  Ax=-0.12, Ay=0.23, Az=9.96 | Gx=1.12, Gy=-1.83, Gz=-0.48 | Pitch=0.01, Roll=0.49
[23:43:54]  Ax=-1.05, Ay=0.17, Az=9.95 | Gx=1.67, Gy=0.91, Gz=-0.72 | Pitch=0.15, Roll=1.12
[23:43:54]  Ax=-0.59, Ay=0.15, Az=9.72 | Gx=1.63, Gy=1.38, Gz=-0.29 | Pitch=0.28, Roll=1.43
[23:43:54]  Ax=-0.82, Ay=-0.15, Az=9.89 | Gx=0.94, Gy=2.20, Gz=-0.84 | Pitch=0.27, Roll=1.80
[23:43:54]  Ax=-0.67, Ay=-0.00, Az=9.87 | Gx=1.68, Gy=0.10, Gz=-0.59 | Pitch=0.24, Roll=2.08
[23:43:54]  Ax=-0.04, Ay=0.34, Az=9.72 | Gx=0.36, Gy=1.79, Gz=-0.75 | Pitch=0.50, Roll=1.92
[23:43:54]  Ax=-0.98, Ay=-0.05, Az=9.97 | Gx=1.74, Gy=-0.50, Gz=-0.42 | Pitch=0.40, Roll=2.36
[23:43:54]  Ax=-0.01, Ay=0.43, Az=9.79 | Gx=0.27, Gy=1.90, Gz=-0.61 | Pitch=0.70, Roll=2.15
[23:43:54]  Ax=-1.11, Ay=-0.00, Az=9.88 | Gx=1.75, Gy=1.02, Gz=-0.48 | Pitch=0.67, Roll=2.65
[23:43:54

In [ ]:
with open("data_imu.txt", "r") as f:
    content = f.read()
    print(content)


In [42]:
with open("data_imu.txt", "w"):
    pass